# Exploratory Data Analysis 
---
*Disclaimer: This project contains text that is profane, vulgar, and offensive due to the nature of the dataset.*

We live in an age where people's lives have become intertwined with their online presence allowing humans to engage with one another on a larger scale than ever before.  However, not all of these interactions are friendly.  People exploit the fact that their identity remains hidden and choose to target one another with unwarranted abuse causing harm to people they will never meet in person.  For our society to truly prosper from the digital age we have to combat this toxic behavior.  This will enable more and more people who are scared of what other people think when they post on social media to engage freely without the fear of being the target of online hate.  

Our task is to improve civility on social media platforms (e.g., Twitter) and online comment forums (e.g., Reddit) by training a model that determines how likely a users comment will make another user leave a conversation.  With our model, we will create a web application that tracks how toxic each social media platform and online comment forum is to bring awareness to this issue and spark initiative to create a toxic free environment for all users.  

This issue can only be solved with the help of everyone encouraging kindness instead of spreading hate.

## Data Description
The dataset used in this project is from the kaggle competition: [Jigsaw Unintended Bias in Toxicity Classification](https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/data).

In 2017 the Civil Comments platform shutdown and released their archive of ~2 million public comments for researchers to study in an effort to improve civility online. Jigsaw funded the annotation of this data by human raters. 

The column `toxicity` is our toxicity label which contains a number between 0-1 denoting the fraction of human labelers that believed the comment would make someone else leave a conversation. 

For our analysis we'll define the comment as toxic (denoted 1) when the value of our target `toxicity` is greater than or equal to 0.5 otherwise we'll assign the instance to the negative class (denoted 0).

There are a lot of additional labels denoting the fraction of human labelers who believed the comment depicted several other sub-toxic labels and whether specific identity attributes were mentioned in the comment. These subtype attributes are:
* severe_toxicity
* obscene
* threat
* insult
* identity_attack
* sexual_explicit
* male
* female
* transgender
* other_gender
* heterosexual
* homosexual_gay_or_lesbian
* bisexual
* other_sexual_orientation
* christian
* jewish
* muslim
* hindu
* buddhist
* atheist
* other_religion
* black
* white
* asian
* latino
* other_race_or_ethnicity
* physical_disability
* intellectual_or_learning_disability
* psychiatric_or_mental_illness
* other_disability

It was also mentioned that different comments might have the exact same text, but labeled with different targets or sybtype attributes.

These subtype attributes will be **removed** from the training schema because we will not have access to them in a production environment.  However, they will be very resourceful as tags used during error analysis to see where exactly our model is performing poorly on.

### Labeling Schema
As mentioned on Kaggle, each comment was shown to up to 10 human labelers.  The labelers were asked to rate how toxic each comment is. 
* Very Toxic
* Toxic
* Hard to Say
* Not Toxic

The ratings were then aggragated into the `target` column.

The subtype attributes were collected by asking the human labelers whether the attribute was mentioned in the comment.  Then, taking these responses and aggregating them into a value between 0-1 representing the fraction of human labelers who believed the subtype attribute was mentioned in the comment.

*Note: Some comments were shown to more than 10 human labelers because of sampling strategies to increase rating accuracy.*

In [1]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import os
import pandas as pd

import nltk
from sklearn.model_selection import train_test_split
import tensorflow as tf
import textblob
import wordcloud

# Root directory used to navigate tree.
PROJECT_ROOT_DIR = os.path.dirname(os.getcwd())

# Path to save images and figures.
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "reports/figures")
if not os.path.exists(IMAGES_PATH):
    os.makedirs(IMAGES_PATH, exist_ok=True)
    
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    """
    Saves figures in toxic_media/reports/figures.
    
    Parameters
    ----------
    fig_id : str
        Name of the figure to be saved.
    tight_layout : bool, default=True
        Enables padding between figure edge and edges of subplots when set to True.
    fig_extension : {'jpg', 'png', 'svg'}, default='png'
        Figure format.
    resolution : int
        Figure resolution.
    """
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print(f"Saving figure: {fig_id}")
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## Load Data
We're going to start by loading the raw data to get a quick look at it before creating a training set with which we'll use to explore in-depth. As mentioned above, the initial data file has extra columns we will not be using.  Therefore, we'll only load in the target column and the comment text. 

In [2]:
DATA_DIR = os.path.join(PROJECT_ROOT_DIR, "data")

def load_data(path="raw/all_data.csv"):
    """
    Load data into pandas dataframe object.
    
    Parameters
    ----------
    path : str, default=raw/all_data.csv
        Directory and filename in format 'directory/filename.csv'.
    """
    csv_path = os.path.join(DATA_DIR, path)
    
    return pd.read_csv(csv_path)

In [3]:
toxic = load_data()

In [4]:
toxic.head()

,id,comment_text,split,created_date,publication_id,parent_id,article_id,rating,funny,wow,...,white,asian,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability,identity_annotator_count,toxicity_annotator_count
0,1083994,He got his money... now he lies in wait till a...,train,2017-03-06 15:21:53.675241+00,21,NaN,317120,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,67
1,650904,Mad dog will surely put the liberals in mental...,train,2016-12-02 16:44:21.329535+00,21,NaN,154086,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,76
2,5902188,And Trump continues his lifelong cowardice by ...,train,2017-09-05 19:05:32.341360+00,55,NaN,374342,approved,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,63
3,7084460,"""while arresting a man for resisting arrest"".\...",test,2016-11-01 16:53:33.561631+00,13,NaN,149218,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,76
4,5410943,Tucker and Paul are both total bad ass mofo's.,train,2017-06-14 05:08:21.997315+00,21,NaN,344096,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,80


In [5]:
toxic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999516 entries, 0 to 1999515
Data columns (total 46 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   id                                   int64  
 1   comment_text                         object 
 2   split                                object 
 3   created_date                         object 
 4   publication_id                       int64  
 5   parent_id                            float64
 6   article_id                           int64  
 7   rating                               object 
 8   funny                                int64  
 9   wow                                  int64  
 10  sad                                  int64  
 11  likes                                int64  
 12  disagree                             int64  
 13  toxicity                             float64
 14  severe_toxicity                      float64
 15  obscene                         

In [6]:
toxic.isnull().sum() / len(toxic)

id                                     0.000000e+00
comment_text                           5.001210e-07
split                                  0.000000e+00
created_date                           0.000000e+00
publication_id                         0.000000e+00
parent_id                              4.325082e-01
article_id                             0.000000e+00
rating                                 0.000000e+00
funny                                  0.000000e+00
wow                                    0.000000e+00
sad                                    0.000000e+00
likes                                  0.000000e+00
disagree                               0.000000e+00
toxicity                               0.000000e+00
severe_toxicity                        0.000000e+00
obscene                                0.000000e+00
sexual_explicit                        0.000000e+00
identity_attack                        0.000000e+00
insult                                 0.000000e+00
threat      

In [7]:
toxic.describe()

,id,publication_id,parent_id,article_id,funny,wow,sad,likes,disagree,toxicity,...,white,asian,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability,identity_annotator_count,toxicity_annotator_count
count,1.999516e+06,1.999516e+06,1.134709e+06,1.999516e+06,1.999516e+06,1.999516e+06,1.999516e+06,1.999516e+06,1.999516e+06,1.999516e+06,...,448000.000000,448000.000000,448000.000000,448000.000000,448000.000000,448000.000000,448000.000000,448000.000000,1.999516e+06,1.999516e+06
mean,4.065400e+06,4.988997e+01,3.715138e+06,2.810257e+05,2.776687e-01,4.437174e-02,1.089289e-01,2.441188e+00,5.808151e-01,1.029241e-01,...,0.056534,0.011886,0.006151,0.008158,0.001351,0.001117,0.012068,0.001219,1.431667e+00,8.775720e+00
std,2.527563e+06,2.771895e+01,2.451507e+06,1.040778e+05,1.054819e+00,2.458644e-01,4.555570e-01,4.712994e+00,1.854332e+00,1.970386e-01,...,0.215175,0.086906,0.058828,0.042429,0.017461,0.016391,0.089072,0.014114,1.763593e+01,4.331605e+01
min,5.984800e+04,2.000000e+00,6.100600e+04,2.006000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,3.000000e+00
25%,8.565798e+05,2.100000e+01,7.930110e+05,1.600038e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,4.000000e+00
50%,5.340220e+06,5.400000e+01,5.217531e+06,3.319250e+05,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,4.000000e+00
75%,5.955782e+06,5.400000e+01,5.774684e+06,3.662270e+05,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,1.666667e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,6.000000e+00
max,7.194640e+06,1.150000e+02,6.333965e+06,3.995440e+05,1.020000e+02,2.100000e+01,3.100000e+01,3.000000e+02,1.870000e+02,1.000000e+00,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.600000,1.866000e+03,4.936000e+03


In [9]:
toxic["toxicity"].round().value_counts() / len(toxic)

0.0    0.941048
1.0    0.058952
Name: toxicity, dtype: float64

### Summary
After our initial look into the data we found a few things to take note of:
* We're missing text in a few instances (handle this in preprocessing).
* Our target column `toxicity` is heavily skewed towards 0 meaning we'll have to deal with a class imbalance when it comes to modeling (~5% of comments are toxic).
* Many of the subtype attributes regarding identities are 77% null meaning we might not be able to make use of these tags during error analysis.  However, the sub-toxic attributes are complete and will be helpful understanding where our model is misclassifying toxic comments. 

## Test Set
Before we dive deep into the dataset we need to create a test set.  Our test set will be 5% (~100,000 instances) which is plenty big enough to provide accurate statistics in our modeling phase. Since we plan on building deep learning models which benefit greatly from large amounts of data this will allow us to allocate 95% of our dataset for training.  

Since our dataset is imbalanced we will use stratified sampling to preserve the ratio of classes in the test set.

In [45]:
X = toxic['comment_text']
y = toxic['toxicity'].round()

def create_test(X, y, test_size=0.05):
    """
    Create holdout data with stratified sampling techniques.
    
    Parameters
    ----------
    X : array-like of shape (n_samples, n_features)
        Raw feature set to be split into training and test set.
    test_size : float, default=0.05
        Decimal value representing percentage of dataset used to create holdout set.
        
    Notes
    -----
    This function only creates the test set if it has not been created yet.  
    """
    # Check if test set exists.
    TEST_PATH = os.path.join(DATA_DIR, "interim/test.csv")
    if not os.path.exists(TEST_PATH):
        
        # Train test split with stratified sampling of the target.
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
                                                            stratify=y)
        
        # Combine target and text into train and test sets.
        train = pd.concat([X_train, y_train], axis=1)
        test = pd.concat([X_test, y_test], axis=1)
        
        # Save train and test set in directory ~/data/interim
        path = os.path.join(DATA_DIR, "interim")
        train.to_csv(os.path.join(path, "train.csv"), index=False)
        test.to_csv(os.path.join(path, "test.csv"), index=False)

create_test(X, y)

## Data Exploration

In [46]:
toxic = load_data(path="interim/train.csv")